In [2]:
import numpy as np 
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import xgboost
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import chardet
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from IPython.display import clear_output
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
# Import data from csv and create a database (including table) and a dataframe.

Path('maternity_risk_data.db').touch()
#con = sqlite3.connect("/kaggle/working/maternity_risk_data.db")
con = sqlite3.connect("maternity_risk_data.db")
cur = con.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS risk_tb (Age, SystolicBP, DiastolicBP, BS, BodyTemp, HeartRate, RiskLevel)''')
df = pd.read_csv('Maternal Health Risk Data Set.csv')
index = [i + 1 for i in df.index]
df.insert(1, 'index_col', index) # Add a column for index since it was missing from original dataset. 
df.to_sql('risk_tb', con, if_exists='replace', index=False)
pregnancy_risk = pd.read_sql('''SELECT * FROM risk_tb''',con)
pregnancy_risk

,Age,index_col,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,1,130,80,15.0,98.0,86,high risk
1,35,2,140,90,13.0,98.0,70,high risk
2,29,3,90,70,8.0,100.0,80,high risk
3,30,4,140,85,7.0,98.0,70,high risk
4,35,5,120,60,6.1,98.0,76,low risk
...,...,...,...,...,...,...,...,...
1009,22,1010,120,60,15.0,98.0,80,high risk
1010,55,1011,120,90,18.0,98.0,60,high risk
1011,35,1012,85,60,19.0,98.0,86,high risk
1012,43,1013,120,90,18.0,98.0,70,high risk


In [3]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(pregnancy_risk):
    # Drop column: 'index_col'
    pregnancy_risk = pregnancy_risk.drop(columns=['index_col'])
    return pregnancy_risk

pregnancy_risk_clean = clean_data(pregnancy_risk.copy())
pregnancy_risk_clean.head() 

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk


In [5]:
# View the first few rows of the DataFrame
print(pregnancy_risk_clean.head())

# Summary statistics for numerical columns
print(pregnancy_risk_clean.describe())

# Information on the DataFrame including the data types of each column
print(pregnancy_risk_clean.info())


   Age  SystolicBP  DiastolicBP    BS  BodyTemp  HeartRate  RiskLevel
0   25         130           80  15.0      98.0         86  high risk
1   35         140           90  13.0      98.0         70  high risk
2   29          90           70   8.0     100.0         80  high risk
3   30         140           85   7.0      98.0         70  high risk
4   35         120           60   6.1      98.0         76   low risk
               Age   SystolicBP  DiastolicBP           BS     BodyTemp  \
count  1014.000000  1014.000000  1014.000000  1014.000000  1014.000000   
mean     29.871795   113.198225    76.460552     8.725986    98.665089   
std      13.474386    18.403913    13.885796     3.293532     1.371384   
min      10.000000    70.000000    49.000000     6.000000    98.000000   
25%      19.000000   100.000000    65.000000     6.900000    98.000000   
50%      26.000000   120.000000    80.000000     7.500000    98.000000   
75%      39.000000   120.000000    90.000000     8.000000    9

In [6]:
pregnancy_risk_clean

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk
...,...,...,...,...,...,...,...
1009,22,120,60,15.0,98.0,80,high risk
1010,55,120,90,18.0,98.0,60,high risk
1011,35,85,60,19.0,98.0,86,high risk
1012,43,120,90,18.0,98.0,70,high risk


In [7]:
pregnancy_risk_clean.columns

Index(['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate',
       'RiskLevel'],
      dtype='object')

In [8]:
risk = pregnancy_risk_clean.copy()

In [11]:
# For numerical columns
for col in risk.select_dtypes(include=['float64', 'int64']):
    risk[col].fillna(risk[col].mean(), inplace=True)

# For categorical columns
for col in risk.select_dtypes(include=['object']):
    risk[col].fillna(risk[col].mode()[0], inplace=True)


In [12]:
risk

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel,RiskScore
0,25,130,80,15.0,98.0,86,high risk,3
1,35,140,90,13.0,98.0,70,high risk,3
2,29,90,70,8.0,100.0,80,high risk,3
3,30,140,85,7.0,98.0,70,high risk,3
4,35,120,60,6.1,98.0,76,low risk,1
...,...,...,...,...,...,...,...,...
1009,22,120,60,15.0,98.0,80,high risk,3
1010,55,120,90,18.0,98.0,60,high risk,3
1011,35,85,60,19.0,98.0,86,high risk,3
1012,43,120,90,18.0,98.0,70,high risk,3


In [13]:
# Define a mapping from risk level to number
risk_mapping = {"high risk": 3, "mid risk": 2, "low risk": 1}

# Replace the risk level categories with numbers
risk["RiskScore"] = risk["RiskLevel"].replace(risk_mapping)


In [14]:
risk

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel,RiskScore
0,25,130,80,15.0,98.0,86,high risk,3
1,35,140,90,13.0,98.0,70,high risk,3
2,29,90,70,8.0,100.0,80,high risk,3
3,30,140,85,7.0,98.0,70,high risk,3
4,35,120,60,6.1,98.0,76,low risk,1
...,...,...,...,...,...,...,...,...
1009,22,120,60,15.0,98.0,80,high risk,3
1010,55,120,90,18.0,98.0,60,high risk,3
1011,35,85,60,19.0,98.0,86,high risk,3
1012,43,120,90,18.0,98.0,70,high risk,3


In [20]:
# # One hot encoding by pandas
# ohe=pd.get_dummies(risk, drop_first=False)

In [25]:
dummies = pd.get_dummies(pregnancy_risk_clean)
print(dummies.columns)

from sklearn.model_selection import train_test_split

# Set age as feature
X=dummies.drop(['RiskLevel_high risk', 'RiskLevel_low risk', 'RiskLevel_low risk'], axis=1)
y=dummies[['RiskLevel_high risk', 'RiskLevel_low risk', 'RiskLevel_low risk']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Index(['Age', 'SystolicBP', 'DiastolicBP', 'BS', 'BodyTemp', 'HeartRate',
       'RiskLevel_high risk', 'RiskLevel_low risk', 'RiskLevel_mid risk'],
      dtype='object')


In [26]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the features
X = scaler.fit_transform(X)


In [18]:
from sklearn.model_selection import train_test_split

# Split the data, let's use 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize base classifier
base_classifier = LogisticRegression(max_iter=10000)

# Initialize the multi-output classifier
model = MultiOutputClassifier(base_classifier)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Generate classification report for each output
for i in range(y_test.shape[1]):
    print(f"Classification report for output {y_test.columns[i]}:")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))


Classification report for output RiskLevel_high risk:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       156
           1       0.89      0.83      0.86        47

    accuracy                           0.94       203
   macro avg       0.92      0.90      0.91       203
weighted avg       0.94      0.94      0.94       203

Classification report for output RiskLevel_low risk:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94       123
           1       0.88      0.94      0.91        80

    accuracy                           0.93       203
   macro avg       0.92      0.93      0.92       203
weighted avg       0.93      0.93      0.93       203

Classification report for output RiskLevel_low risk:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94       123
           1       0.88      0.94      0.91        80

    accuracy          